In [7]:
import os
import scipy.io
import numpy as np

import xgboost as xgb
import lightgbm as lgb
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.svm import SVC   #for Support Vector Machine (SVM) Algorithm

from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm


import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.preprocessing.sequence import TimeseriesGenerator

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv1D, Conv2D, MaxPooling1D

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
import os
import scipy.io
import numpy as np

def load_data(folder_path, exclude_filetypes=[]):
    all_epochs = []
    all_triggers = []
    all_labels = []
    all_stimuli = []
    all_phase = []
    all_subjects = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".mat"):
                filepath = os.path.join(root, file)
                print(file)
                
                # Check if the file should be excluded based on the exclude_filetypes list
                if any(filetype in file for filetype in exclude_filetypes):
                    print(f'Excluding file types {file}')
                    continue

                mat = scipy.io.loadmat(filepath)
                # Replace 'epoch' with the actual key in your .mat files
                data = mat['epoch']  # Shape: (n_channels, n_times, n_epochs)
                
                if data is None:
                    print(f'File {filepath} is empty')
                    continue
                print(data.shape)
                if len(data.shape) < 3:
                    data = np.expand_dims(data,axis=2)
                # Get the trigger type from the filename
                trigger_type = int(file.split('_type')[-1].split('.')[0])
                
                # Create a list of triggers based on the number of epochs
                triggers = [trigger_type] * data.shape[2]
                
                # Get the label from the folder name
                label = 0 if 'Not_Blinded' in root else 1
                
                # Append data to the lists
                all_epochs.append(data)
                all_triggers.extend(triggers)
                all_labels.extend([label] * data.shape[2])
                #stimulus, phase = get_stimulus_and_phase(filepath) # NORMAL IB
                #stimulus, phase = get_stimulus_and_phase_oddball(filepath) # ODDBALL
                stimulus, phase = get_stimulus_and_phase_ERN(filepath) # ODDBALL
                all_stimuli.extend([stimulus] * data.shape[2])
                all_phase.extend([phase] * data.shape[2])
                all_subjects.extend([get_subject_number(filepath)] * data.shape[2])
                
    # Concatenate lists into numpy arrays
    all_epochs = np.concatenate(all_epochs, axis=2)  # Shape: (n_channels, n_times, total_epochs)
    all_triggers = np.array(all_triggers)  # Shape: (total_epochs,)
    all_labels = np.array(all_labels)  # Shape: (total_epochs,)

    return all_epochs, all_triggers, all_labels, all_phase, all_stimuli, all_subjects

# Assuming get_stimulus_and_phase and get_subject_number are defined elsewhere


In [10]:
import re

def get_subject_number(file_name):
    match = re.search(r'IB_Sub_(\d+)_', file_name)
    if match:
        #print(int(match.group(1)))
        return int(match.group(1))
        
    else:
        #print("Found None Subj Num")
        return None  # or handle the case where no subject number is found
        
        #print(int(match.group(1)))


In [15]:
def get_stimulus_and_phase_ERN(file_name):
    stimulus = ""
    phase = ""

# '12180','13180','22180','23180'
    if '12180' in file_name:
        stimulus = 'correct'
        phase = 'invisible'
    elif '13180' in file_name:
        stimulus = 'error'
        phase = 'invisible'
    elif '22180' in file_name:
        stimulus = 'correct'
        phase = 'visible'
    elif '23180' in file_name:
        stimulus = 'error'
        phase = 'visible'
        
    return stimulus, phase

# Example usage
#stimulus, phase = get_stimulus_and_phase(file_name)
#
# 
# 11132

In [12]:
def get_stimulus_and_phase_oddball(file_name):
    stimulus = ""
    phase = ""

    if '11132' in file_name:
        stimulus = 'same'
        phase = 'invisible'
    elif '11133' in file_name:
        stimulus = 'different'
        phase = 'invisible'
    elif '21132' in file_name:
        stimulus = 'same'
        phase = 'visible'
    elif '21133' in file_name:
        stimulus = 'different'
        phase = 'visible'
    elif '31132' in file_name:
        stimulus = 'same'
        phase = 'reported'
    elif '31133' in file_name:
        stimulus = 'different'
        phase = 'reported'
        
    elif '12132' in file_name:
        stimulus = 'error_same'
        phase = 'invisible'
    elif '12133' in file_name:
        stimulus = 'error_different'
        phase = 'invisible'
    elif '22132' in file_name:
        stimulus = 'error_same'
        phase = 'visible'
    elif '22133' in file_name:
        stimulus = 'error_different'
        phase = 'visible'
    elif '32132' in file_name:
        stimulus = 'error_same'
        phase = 'reported'
    elif '32133' in file_name:
        stimulus = 'error_different'
        phase = 'reported'
        
    #print(f"Stimulus: {stimulus}, Phase: {phase}")

    return stimulus, phase

# Example usage
#stimulus, phase = get_stimulus_and_phase(file_name)
#
# 
# 11132

In [13]:
def get_stimulus_and_phase(file_name):
    stimulus = ""
    phase = ""

    if 'type210' in file_name:
        stimulus = 'face'
        phase = 'invisible'
    elif 'type220' in file_name:
        stimulus = 'face'
        phase = 'visible'
    elif 'type230' in file_name:
        stimulus = 'face'
        phase = 'reported'
    elif 'type211' in file_name:
        stimulus = 'house'
        phase = 'invisible'
    elif 'type221' in file_name:
        stimulus = 'house'
        phase = 'visible'
    elif 'type231' in file_name:
        stimulus = 'house'
        phase = 'reported'
    elif 'type212' in file_name:
        stimulus = 'noise'
        phase = 'invisible'
    elif 'type222' in file_name:
        stimulus = 'noise'
        phase = 'visible'
    elif 'type232' in file_name:
        stimulus = 'noise'
        phase = 'reported'
        
    #print(f"Stimulus: {stimulus}, Phase: {phase}")

    return stimulus, phase

# Example usage
#stimulus, phase = get_stimulus_and_phase(file_name)
#


In [6]:

# Load data from both folders
blinded_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Epochs_full"
not_blinded_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Epochs_full"

blinded_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Epochs_full"

disk_folder = "D:\\Triangulation\\Reed\\Raw\\Subjects\\DONE\\DONE_ICA\\DONE_ICA\\DONE_ICA\\Disk_Trigger_added\\Epochs_full\\Blinded"

ERN_folder = "D:\\Triangulation\\Reed\\Raw\\ERN\\Epochs_full\\Epochs_full"


In [16]:
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(ERN_folder)

IB_Sub_111_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA07_EpRj0271_type12180.mat
(64, 700, 256)
IB_Sub_111_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA07_EpRj0271_type13180.mat
(64, 700, 65)
IB_Sub_111_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA07_EpRj0271_type22180.mat
(64, 700, 431)
IB_Sub_111_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA07_EpRj0271_type23180.mat
(64, 700, 117)
IB_Sub_112_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA10_EpRj0007_type12180.mat
(64, 700, 537)
IB_Sub_112_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA10_EpRj0007_type13180.mat
(64, 700, 34)
IB_Sub_112_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA10_EpRj0007_type22180.mat
(64, 700, 546)
IB_Sub_112_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA10_EpRj0007_type23180.mat
(64, 700, 27)
IB_Sub_113_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA12_EpRj0086_type12180.mat
(64, 700, 518)
IB_Sub_113_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA12_EpRj0086_type13180.mat
(64, 700, 20)
IB_Sub_113_500hz_HM64_Hp1Lp55NF60_TagERR_ep_ICA12_EpRj0086_type22180.mat
(64, 700, 553)
IB_Sub_113_500hz_HM64_Hp1Lp55NF60_Ta

In [17]:
import pickle
with open('IB_eeg_data_ERN_BLINDED.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [34]:
# DISK DATA SAVE
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(disk_folder)

IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type11132.mat
(64, 750, 184)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type11133.mat
(64, 750, 186)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type12132.mat
(64, 750, 9)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type12133.mat
(64, 750, 7)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type21132.mat
(64, 750, 196)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type21133.mat
(64, 750, 170)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type22132.mat
(64, 750, 8)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type22133.mat
(64, 750, 14)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type31132.mat
(64, 750, 183)
IB_Sub_11_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj04_EpRj0006DONE_Disc_type31133.mat
(64, 750, 196)
IB_Sub_

In [36]:
import pickle
with open('IB_eeg_data_DISK_BLINDED.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [7]:

epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type22','type23'])


In [8]:
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type22','type23'])
import pickle
# Save the data
with open('IB_eeg_data_BLINDED_phase1.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type210.mat
IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type211.mat
IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type212.mat
IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type220.mat
Excluding file types IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type220.mat
IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type221.mat
Excluding file types IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type221.mat
IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type222.mat
Excluding file types IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type222.mat
IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type230.mat
Excluding file types IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0881DONE_type230.mat
IB_Sub_93_500hz_HM63_Hp1Lp55NF60_LEfix_Ep_ICA_IcRj05_EpRj0

In [10]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type21','type23'])
import pickle
# Save the data
with open('IB_eeg_data_BLINDED_phase2.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [13]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(blinded_folder,['type21','type22'])
import pickle
# Save the data
with open('IB_eeg_data_BLINDED_phase3.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [9]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(not_blinded_folder,['type22','type23'])
import pickle
# Save the data
with open('IB_eeg_data_NOTBLINDED_phase1.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [11]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(not_blinded_folder,['type21','type23'])
import pickle
# Save the data
with open('IB_eeg_data_NOTBLINDED_phase2.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [12]:
epochs_blinded = []
epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b = load_data(not_blinded_folder,['type21','type22'])
import pickle
# Save the data
with open('IB_eeg_data_NOTBLINDED_phase3.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

In [10]:
epochs_blinded = []

In [11]:
epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n  = load_data(not_blinded_folder)

In [12]:
    # Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n), f)

In [8]:

epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n  = load_data(not_blinded_folder)

# Combine data from both folders
epochs = np.concatenate([epochs_blinded, epochs_not_blinded], axis=2)
triggers = np.concatenate([triggers_blinded, triggers_not_blinded])
labels = np.concatenate([labels_blinded, labels_not_blinded])
phase = np.concatenate([phase_b, phase_n])
stim = np.concatenate([stimuli_b, stimuli_n])
subj = np.concatenate([subjects_b, subjects_n])

print("Epochs shape:", epochs.shape)
print("Triggers shape:", triggers.shape)
print("Labels shape:", labels.shape)

print("Epochs shape:", stim.shape)
print("Triggers shape:", phase.shape)
print("Labels shape:", subj.shape)

import pickle


# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n), f)

# Save the data
# with open('IB_eeg_data_NEW.pkl', 'wb') as f:
#     pickle.dump((epochs, triggers, labels, phase, stim, subj), f)


Epochs shape: (64, 1250, 291272)
Triggers shape: (291272,)
Labels shape: (291272,)
Epochs shape: (291272,)
Triggers shape: (291272,)
Labels shape: (291272,)


MemoryError: 

In [10]:

# # Combine data from both folders
# epochs = np.concatenate([epochs_blinded, epochs_not_blinded], axis=2)
# triggers = np.concatenate([triggers_blinded, triggers_not_blinded])
# labels = np.concatenate([labels_blinded, labels_not_blinded])
# phase = np.concatenate([phase_b, phase_n])
# stim = np.concatenate([stimuli_b, stimuli_n])
# subj = np.concatenate([subjects_b, subjects_n])

# print("Epochs shape:", epochs.shape)
# print("Triggers shape:", triggers.shape)
# print("Labels shape:", labels.shape)

# print("Epochs shape:", stim.shape)
# print("Triggers shape:", phase.shape)
# print("Labels shape:", subj.shape)

# import pickle

# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_blinded, triggers_blinded, labels_blinded, phase_b, stimuli_b, subjects_b), f)

# Save the data
with open('IB_eeg_data_NEW.pkl', 'wb') as f:
    pickle.dump((epochs_not_blinded, triggers_not_blinded, labels_not_blinded, phase_n, stimuli_n, subjects_n), f)

MemoryError: 